<a href="https://colab.research.google.com/github/mikolaura/impleting_basic_rag_pipeline/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install all libraries from pip

In [ ]:
%pip install langchain langchain-google-genai faiss-cpu  langchain-community huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 

In [ ]:
%pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

## Loading all libraries into a notebook

In [ ]:
import os
import getpass
from datasets import load_dataset
import pandas as pd
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
import faiss
import unittest

# Create vector store and loading dataset from scratch

**You also can use already created faiss vector store**

*for this move down to `Alternative: Load vector store`*


## Loading and structuring dataset

`To done this part you would need hugging face api token`




In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `ds` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ds`


In [ ]:
ds = load_dataset("uran050311/wikipedia_rag")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

text_with_empty_vector.csv:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
# Converting into pandas data frame and preprocesing data
train_data = pd.DataFrame.from_dict((ds['train']))
texts = train_data['text'].tolist()


## Vector store

***You need google api key for this part***

In this part:

* We create document from text
* We split documents
* We create vector store
* We add data to vector store

In [ ]:
# Preprocess text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
documents = text_splitter.create_documents(texts)
splited_documents = text_splitter.split_documents(documents)

### Create embeddings. Convert from text to embeddings

In [ ]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass()

··········


In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
def from_text_to_embeddings(embeddings, text):
    return embeddings.embed_query(text)

### Setup Faiss

In [ ]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
vector_store.add_documents(documents=splited_documents)

['9937aed6-f605-4400-a9a3-39e46bc43873',
 '8f206c68-4bcd-4047-83f6-355787d22fd0',
 '849e96ec-188c-4856-8d8b-81941f694959',
 '44c44628-9ccd-460b-b5cc-e53ce6e3b426',
 '8c2cbb40-598e-4079-a8a3-5991f31c2467',
 '0f4436d1-e90f-41b1-94e3-78940f8d05f2',
 'b8abc7fa-9d1c-44d2-a1b4-7d22c10655db',
 'e2eae147-8ec5-440f-9046-99ebd44a6317',
 'abf6c20d-04f4-4397-b816-08fa4bfb1bc9',
 'e83d9496-a128-448f-a6ad-6b92b8315bfc',
 '9100e9e1-8a16-4620-8299-06f76444a187',
 '6ccec92d-269c-4b4b-925c-dc837d14d00f',
 '39fb1a57-7359-49df-b1a5-f909a7fb5011',
 'ef52569c-78a5-49a1-a615-aeb19a07289c',
 '0c6f93dc-9147-4184-a742-4cf907f86e11',
 'a63367a1-efa3-4be3-a2db-f5fd97cdfbaf',
 '1736d6fd-9f93-4266-919e-7f5332d53671',
 '1bf2455b-5ed8-4872-a36a-06f249451f4e',
 'bbd66543-d5e8-486d-b64c-30a1f408f555',
 'ea32b458-04b6-4166-96aa-ae146ebd7ce8',
 '44d36cd8-b521-48cd-a261-8489714da626',
 'b6a11e4d-e85d-4a38-b304-e3694d02b205',
 'b6900e08-8e93-4bfe-95d1-081702dfcb05',
 '83975e3d-031c-48bc-a925-4db4e5d565b6',
 'c337c277-2dc1-

## Creating LLM and retriever

This part:

* Initializing LLM
* Createing retriver
* Creating rag chain
* And make first prediction

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    model_kwargs = {
        "seed": 42
    }
    )

In [ ]:
retriever = vector_store.as_retriever()
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
result = rag_chain.run("What is ASCII")
print("Answer:", result)

<ipython-input-13-5561a7a1e165>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain.run("What is ASCII")


Answer: ASCII, abbreviated from American Standard Code for Information Interchange, is a character encoding standard for electronic communication. ASCII codes represent text in computers, telecommunications equipment, and other devices. It encodes 128 specified characters into seven-bit integers.


## Unit testing

In [ ]:
class TestStringMethods(unittest.TestCase): # not all unit test would work because llm can change it result
  def test_what_is_ascii(self):
      self.assertIn(rag_chain.run("What is ASCII"), "ASCII, abbreviated from American Standard Code for Information Interchange, is a character encoding standard for electronic communication. ASCII codes represent text in computers, telecommunications equipment, and other devices. It encodes 128 specified characters into seven-bit integers.")
  def test_where_is_andorra(self):
      self.assertIn(rag_chain.run("Where is Andorra"), "Andorra is a landlocked country in the eastern Pyrenees mountains, bordered by France to the north and Spain to the south.")

In [ ]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 1.482s

OK


## Saving dataset with vector

In [ ]:
rows = []

for i in range(len(texts)):
    vector = []
    # This code work, but because gemini cannot process that much data it would take forever to run

      # for q in range(100, len(texts[i])-100, 100):

      #   vec = embeddings.embed_query(texts[i][q-100:q])
      #   vector.extend(vec)
    rows.append({
        "id": i,
        "title": train_data['title'].tolist()[i],
        "text": texts[i],
        "vector": vector
    })

df = pd.DataFrame(rows)
df.to_csv("text_with_empty_vector.csv", index=False)

## Saving faiss vector store

In [ ]:
vector_store.save_local("my_faiss_index")

# Alternative: Load vector store

***For this you would need google api key***

In [ ]:
!wget https://github.com/mikolaura/faiss_index/raw/refs/heads/main/my_faiss_index.zip

--2025-06-07 12:56:01--  https://github.com/mikolaura/faiss_index/raw/refs/heads/main/my_faiss_index.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mikolaura/faiss_index/refs/heads/main/my_faiss_index.zip [following]
--2025-06-07 12:56:01--  https://raw.githubusercontent.com/mikolaura/faiss_index/refs/heads/main/my_faiss_index.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9108732 (8.7M) [application/zip]
Saving to: ‘my_faiss_index.zip’

my_faiss_index.zip  100%[===================>]   8.69M  --.-KB/s    in 0.07s   

2025-06-07 12:56:02 (132 MB/s) - ‘my_faiss_index.zip’ saved [9108732/

In [ ]:
!unzip my_faiss_index.zip

Archive:  my_faiss_index.zip
   creating: my_faiss_index/
 extracting: my_faiss_index/index.faiss  
 extracting: my_faiss_index/index.pkl  


## OPTIONAL: USING CLI INTERFACE

In [ ]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass()

In [ ]:
#Loading embeddings and vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.load_local("my_faiss_index",
                                embeddings,
                                allow_dangerous_deserialization=True
                                )

In [ ]:
# Init chat model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
# Creating retriver and rag chain
retriever = vector_store.as_retriever()
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [ ]:
# Predicting on own input
result = rag_chain.run(input())
print("Answer:", result)

Answer: Oscar is mainly known as the Academy Awards.


## OPTIONAL: CREATE WEB-GUI USING STREAMLIT

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.7 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import os
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
import faiss
import streamlit as st
with st.sidebar:
    google_gemini_api_key = st.text_input(
        "Google Gemini API Key",
        key="api_key",
        type='password'
    )

st.title('RAG')

if google_gemini_api_key:
    os.environ['GOOGLE_API_KEY'] = google_gemini_api_key
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.load_local("my_faiss_index",
                                embeddings,
                                allow_dangerous_deserialization=True
                                )
    # Init chat model
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    # Creating retriver and rag chain
    retriever = vector_store.as_retriever()
    rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
    text = st.text_input(
        "Write your question",
        key="text"
    )
    # Predicting on own input
    if text:
        result = rag_chain.run(text)
        print("Answer:", result)
        st.write(result)

Overwriting app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸
up to date, audited 23 packages in 1s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠸

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.127.106.182

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://old-walls-call.loca.lt
^C
